In [8]:
# This script is to generate a helper file which help to transform the Solid data into sample pairs
import pandas as pd
import csv
import os
import glob

In [9]:
# create a csv file with columns name as video_path,start,end,label.  Adjust the path if needed
# This is to calculate the arousal value, generate the label for each time windows, and store the info about each sample pairs. 
# This file will help prepare the dataset in later parts
with open('solid_pairs.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the column names, 'start' is the start time, 'end' is the end time
    writer.writerow(['video_path', 'start', 'end', 'label'])






In [10]:

# Get the path to the folder where the videos are stored, adjust it if needed
folder_path = './Solid/videos'

# Find all files in the folder
files = glob.glob(os.path.join(folder_path, '*'))

# Get the names of the files
file_names = [os.path.basename(file) for file in files]



In [11]:
gameplay_info = pd.read_csv('clean_data.csv')

C:\Users\XIA Yi\AppData\Local\Temp\ipykernel_11736\2248557627.py:1: DtypeWarning: Columns (23,24,98,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  gameplay_info = pd.read_csv('clean_data.csv')


In [12]:
# This is to generate entries about 3-second time windows in Solid from the clean_data.csv
down = 0
up = 0
total = 0
for file in file_names:
  file_path = str(folder_path)+'/'+str(file)
  print("processing:",file)
  file_name = file[0:-5]
  id_list = file_name.split('_solid_')

  # Find the entries with Solid game sessions
  df1 = gameplay_info.loc[(gameplay_info['[control]player_id']==id_list[0])&(gameplay_info['[control]session_id']==id_list[1])]
  
# Four data per 1 second, interval step is to 4, i represents data index, we want to get the first index in each second
  for i in range(0,len(df1),4):
    # If there is no overlap, the first data index is i at the beginning of every 6 seconds(one sample pair) and the last data index is i+23;
    # i+24 is then the index of the first data of the next 6 seconds. 
    if ((i+23)<len(df1)):
      # Here, To calculate start time and end time of each time window
      start = float(i)
      start = start/4.0
      end = float(i+24)
      end = end/4.0

      # Calculate the arousal value of the corresponding time window based on the time and index.
      df2 = df1.iloc[i:i+24]
      arousal_1 = 0
      arousal_2 = 0

      # The index range for the first 3 seconds is 0-11, including 11, for a total of 12 entries of data
      for j in range(0,12):
        df3 = df2.iloc[j]
        arousal = df3['[output]arousal']
        arousal_1 = arousal_1+arousal
      # average
      arousal_1 = arousal_1/12
      #The index range for the last 3 seconds is 12-23
      for k in range(12,24):
        df3 = df2.iloc[k]
        arousal = df3['[output]arousal']
        arousal_2 = arousal_2+arousal
      arousal_2 = arousal_2/12
      # keep labels simple, arousal unchanging-->"same", arousal decreasing-->"down", arousal increasing-->"up"
      arousal_change = "same"
      if(arousal_1>arousal_2):
        arousal_change = "down"
        down = down+1
      if(arousal_1<arousal_2):
        arousal_change = "up"
        up = up+1
      total = total + 1

      with open('solid_pairs.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([file_path, start, end, arousal_change])

  # As two pairs cannot extract corresponding frames from videos, then removed in later process
  # print("down:",down,"; up:",up, "total:", total)









processing: 018B0D8D-B5EC-09D6-90ED-550058EBFEAF_solid_015529D9-2867-69C8-9C53-CFA6FDCE54A5.webm
processing: 06F9F8CE-9696-23CA-25B5-000AA4A793BA_solid_AF582C53-94D1-BF80-0EDE-F9DC48B90662.webm
processing: 09FCECC4-F94E-ED69-5E9B-3223B590A149_solid_F7DAF2BC-D1AB-011D-4C21-17C1CC3C428F.webm
processing: 0A221ADF-C99E-307E-7B65-92EF9DABA6C4_solid_88462BA0-2404-DDF4-E18E-781452DB204B.webm
processing: 0A76B59A-F1FF-F3F5-A3CA-2386FA8EF308_solid_0DC45818-977E-9063-C15B-1400E3A8242E.webm
processing: 0AD33010-1AC1-E9FB-2FD4-313E3DF4692D_solid_C7996D9A-95E7-750D-5D93-B6A050CCD72A.webm
processing: 0B5863F1-A482-C95D-EA42-9183B5BDFA12_solid_A31BB372-243F-24EA-2D44-47D7B2979995.webm
processing: 0D759376-3F10-1BEB-7205-E91B8BF17984_solid_31470F96-622D-9F1C-2ADD-BC1D210AC88C.webm
processing: 0F3B2D98-51FE-5265-3746-A0763DE31C98_solid_A4BAC22C-D34C-8AB9-3359-2DD2A75FBD2A.webm
processing: 10FFF83C-FD48-DCA7-E554-491C3C1E3EC6_solid_AA34C60D-0D0C-D831-9832-68326266106E.webm
processing: 14027B3A-0AD6-CBA3